## Пакет rgbif

[Краткое описание пакета на портале GBIF](https://www.gbif.org/tool/81747/rgbif)  
[Описание пакета в проект CRAN](https://cran.r-project.org/web/packages/rgbif/rgbif.pdf)  
[краткое описание пакета в официальном репозитории CRAN](https://cran.r-project.org/web/packages/rgbif/index.html)  
[Руководство к актуальной версии 3.3.0 от 8 сентября 2020 года  - ENGLISH](https://cran.r-project.org/web/packages/rgbif/rgbif.pdf)

In [ ]:
install.packages("rgbif")

Пакет **rgbif** также является часть другого пакета [**spocc (SPesies OCCurrence)**](https://github.com/ropensci/spocc),  
который позволяет получать находки видов сразу из нескольких источников данных:
1. [Global Biodiversity Information Facility (GBIF)](https://www.gbif.org/)
2. [Berkeley Ecoengine](https://ecoengine.berkeley.edu/)
3. [iNaturalist](https://www.inaturalist.org/)
4. [VertNet](https://www.vertnet.org/)
5. [Biodiversity Information Serving Our Nation (BISON)](https://bison.usgs.gov/)
6. [eBird](https://ebird.org/)
7. [iDigBio](https://www.idigbio.org/)
8. [Ocean Biodiversity Information System](https://obis.org/)
9. [Atlas of Living Australia](https://www.ala.org.au/)

Разработка началась в 2016 году, актуальная версия пакета **spocc** (1.1.0) вышла 31 июля 2020 года.  
Пакет **spocc** является частью проекта [**rOpenSci**](https://ropensci.org/), направленного на развитие технической инфраструктуры для работы с открытыми данными

In [ ]:
install.packages("spocc")

In [ ]:
library(rgbif)

Пакет работатает с программным интерфейсом (API) портала GBIF и позволяет автоматизировать извлечение необходимой информации. Во многих случаях это будет быстрее и удобнее, чем через интерфейс GBIF.org.
Более 50 функций пакета rgbif позволяют искать информацию о таксонах; запрашивать информацию об организациях, публикующих данные; получать сводную и подробную информацию о находках, а также строить растровые схемы пространственного распределения находок того или иного вида или группы видов.

Начнем с простого - запросим сведения о числе находок, индексированных в информационной системе. Для этого есть функция `occ_count()`, если аргументы не задавать - получим общее число находок.

In [ ]:
occ_count()

Из них имеющих географическую привязку:

In [ ]:
occ_count(georeferenced=TRUE)

% геопривязанных находок

In [ ]:
total = occ_count()
georef = occ_count(georeferenced=TRUE)
round(georef / total,4)*100

Находок для территории России

In [ ]:
occ_count(country = 'RU')

% геопривязанных находок из России

In [ ]:
total = occ_count(country = 'RU')
georef = occ_count(georeferenced=TRUE,country = 'RU')
round(georef / total,4)*100

Построим график публикационной активности (по числу находок) по годам.  
Для этого используем возможность построения цикла в R.  
Для этого используем аргументы `publishingCountry` и `year`

In [ ]:
yearcount = 0
years = 0
startyear = 1950
currentyear = 2020

for(i in startyear:currentyear) {
    occurs = occ_count(country = 'RU', year = i)  
    yearcount[i-startyear] = occurs
    years[i-startyear] = i
}

In [ ]:
plot(yearcount, type = 'l', col = 'green', ylab = 'number of occurrences')
axis(2, labels = years)

Попробуйте сделать то же самое для других стран. Двузначный код страны следует указывать согласно международному стандарту [**ISO 3166-1 alpha-2**](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2)

Обычно в GBIF обращаются, чтобы получить сведения о распространении какого-то вида, т.е. получить набор точек для дальнейшего анализа. Для отого необходимо знать идентификатор нужного таксона. Для этого есть функция `name_backbone`. При запросе идентификатора любого таксона желательно указывать ранг таксона.
Список рангов таксонов, которые предусмотрены в GBIF можно получить следующей командой (без аргументов).

In [ ]:
taxrank()

Проверяем название по таксономической базе [GBIF Backbone Taxonomy](https://www.gbif.org/dataset/d7dddbf4-2cf0-4f39-9b2a-bb099caae36c)

In [ ]:
name_backbone('Linnaea borealis', rank = 'species')

Если результат устраивает - сохраняем идентификатор таксона в переменную `spKey`

In [ ]:
spKey = name_backbone('Linnaea borealis', rank = 'species')$usageKey
spKey

Ради интереса можно посмотреть названия вида на национальных языках

In [ ]:
name_usage(key = 5334220, language = 'eng', data = 'vernacularNames')$data

Есть встроенная функция построения растровой карты:

In [ ]:
x <- map_fetch(taxonKey = spKey)
library(raster)
plot(x, xlim = c(-180,180), ylim = c(-90,90))

In [ ]:
name_backbone("Clematis sibirica")

In [ ]:
spKey = name_backbone("Clematis sibirica")$usageKey
spKey

Прежде чем получить список находок, рекомендуется проверить их количество.  
Для наглядности можно построить круговую диаграмму с привязанными и непривязанными находками.

Максимум значений, которые можно получить при помощи функции `occ_search()` и `occ_data()` 100,000.  
Но сначала желательно проверить число записей при помощи функции `occ_count()`

In [ ]:
occ_count(taxonKey = 7277760)
occ_count(taxonKey = 7277760, georeferenced=TRUE)

In [ ]:
pie(c(occ_count(taxonKey = 7277760, georeferenced=TRUE), occ_count(taxonKey = 7277760, georeferenced=FALSE)), 
    main = 'Число находок Clematis sibirica',
labels = c('геопривязанные','без координат'),
      col = c('cyan','bisque'))

In [ ]:
z <- name_lookup(query='Clematis sibirica', rank="species")
z$data

Для того, чтобы выбрать необходимые поля, используем [Darwin Core Quick Reference Guide](https://dwc.tdwg.org/terms/)
Надо указывать limit, так как по умолчанию выдаёт 500 находок

In [ ]:
occurs = occ_search(taxonKey = 7277760,fields='minimal', limit = 10000)$data

In [ ]:
names(occurs)

In [ ]:
nrow(occurs)